<a href="https://colab.research.google.com/github/lcipolina/Conference-Paper-Highlighter/blob/main/Conference_Paper_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download your favourite papers from your favourite conference
A script by Mihai Anca
https://github.com/MihaiAnca13/Conference-Paper-Highlighter

### Tutorial
* The script will prompt you if you want to download the data, input 'y' on the text box

* A list of conferences will be dumped indexed by numbers. Select the conference you want by iserting the corresponding index number on the text box

* The script will ask for confirmation on your selected number

* Lastly, a couple more text box will appear for you to enter the key word to highlight. For example, you can enter the word *reinforcement* for all the reinforcement papers to be highlighted

* The Excel Spreadsheet will be ready on the left pane of the Google Colab, called 'sample.xls'

* Note: if you are running the script multiple times, make sure to close the file or delete it before running.

In [ ]:
!pip install selenium
!pip install openpyxl

In [ ]:
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


In [ ]:
import os
from time import sleep
from selenium.common.exceptions import NoSuchElementException
import requests
from openpyxl import load_workbook, Workbook
from openpyxl.formatting import Rule
from openpyxl.styles import PatternFill
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.styles.colors import COLOR_INDEX


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
#Main script

RED = COLOR_INDEX[2]
BLUE = COLOR_INDEX[4]
GREEN = COLOR_INDEX[3]
YELLOW = COLOR_INDEX[5]
colors = [RED, BLUE, GREEN, YELLOW]


def removeFormatting(ws):
    for key, _ in ws.conditional_formatting._cf_rules.items():
        del ws.conditional_formatting[key.sqref]

    # ws is not the worksheet name, but the worksheet object
    for row in ws.iter_rows():
        for cell in row:
            cell.style = 'Normal'


if not os.path.isfile('sample.xlsx'):
    wb = Workbook()
    wb.save('sample.xlsx')
else:
    wb = load_workbook('sample.xlsx')

# grab the active worksheet
ws = wb.active

removeFormatting(ws)

ans = input('Download list? [y/n]')
if ans == 'y':
    URL = "https://openreview.net/"

    browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options) #for google colab
    #browser = webdriver.Chrome() comment this out to use on google colab
    browser.get(URL)

    # wait for page to load
    sleep(2)

    # select conference
    confs = browser.find_element_by_id('all-venues')
    confs = confs.find_elements_by_tag_name('a')
    for i, c in enumerate(confs):
        print(i, c.text, c.get_attribute('href'))

    id = int(input())
    new_link = confs[id].get_attribute('href')

    browser.get(new_link)

    # navigate the pages
    while True:
        try:
            options = browser.find_element_by_class_name('venues-list').find_elements_by_tag_name('a')
        except NoSuchElementException:
            break
        for i, o in enumerate(options):
            print(i, o.text, o.get_attribute('href'))

        id = int(input())
        new_link = options[id].get_attribute('href')

        browser.get(new_link)

    # grab all the papers!
    id = browser.current_url
    id = id[id.find('id=')+3:]
    links = []
    links.append(f"https://api.openreview.net/notes?invitation={id}/.*/Submission&details=replyCount,invitation,original&offset=0&includeCount=false&limit=5000")
    # links.append(f"https://api.openreview.net/notes?invitation={id}/Paper.*/-/Decision&details=replyCount,invitation,original&offset=0&includeCount=false&limit=5000")
    links.append(f"https://api.openreview.net/notes?invitation={id}/-/Blind_Submission&details=replyCount,invitation,original&offset=0&includeCount=false&limit=5000")

    for link in links:
        resp = requests.get(link).json()
        if 'count' in resp:
            for i, note in enumerate(resp['notes']):
                if 'decision' in note['content'] and note['content']['decision'] == 'Reject':
                    continue
                title = note['content']['title']
                authors = note['content']['authors']
                keywords = note['content']['keywords']

                print(title, authors, keywords)
                ws[f'A{i+1}'] = title
                ws[f'B{i+1}'] = ", ".join(authors)
                ws[f'C{i+1}'] = ", ".join(keywords)

    browser.close()
# Save the file
wb.save("sample.xlsx")

i = 0
ws['I1'] = "Text to highlight (limited to the first 5000 rows):"
for j in range(2, 6):
    ws[f'I{j}'] = ""
while True:
    print(f'Text to highlight{i+1}/4 (empty for skip): ')
    text_to_find = input()
    if text_to_find == '':
        break

    fill = PatternFill(start_color=colors[i], end_color=colors[i], fill_type='solid')
    dxf = DifferentialStyle(fill=fill)
    ws[f'I{2+i}'] = text_to_find
    ws[f'I{2 + i}'].fill = fill
    rule = Rule(type='containsText', operator='containsText', text=text_to_find, stopIfTrue=False)
    rule.dxf = dxf
    rule.formula = [f"NOT(ISERROR(SEARCH(\"{text_to_find}\",A1)))"]

    ws.conditional_formatting.add('A1:D5000', rule)

    i += 1
    if i == 4:
        break

# Save the file
wb.save("sample.xlsx")
wb.close()

Download list? [y/n]n
Text to highlight1/4 (empty for skip): 

